In [1]:
import pandas as pd
from itertools import combinations
import math

# Cargar el nuevo dataset
file_path = 'input/expert_assessment.csv'
df_new = pd.read_csv(file_path)

# Crear la columna 'story_id' combinando 'title_id' y 'synopsis_writer'
df_new['story_id'] = df_new['title_id'] + '_' + df_new['synopsis_writer']

# Definir las columnas de puntuación a considerar
score_columns = [
    '1_attractive_style', '1_attractive_theme', '2_originality_style', 
    '2_originality_theme', '4_creativity_synopsis', '6_anthology', '6_own_voice'
]

# Inicializar una lista para almacenar las comparaciones por pares
pairwise_data = []
ties_count = 0

# Generar comparaciones por pares considerando la suma de las puntuaciones
for user, group in df_new.groupby('username'):
    num_texts = len(group)
    expected_pairs = math.comb(num_texts, 2)
    print(f"Usuario: {user}, Textos: {num_texts}, Pares esperados: {expected_pairs}")
    pairs_count = 0
    for (idx1, text1), (idx2, text2) in combinations(group.iterrows(), 2):
        # Calcular la suma de puntuación para cada historia
        score_text1 = text1[score_columns].sum()
        score_text2 = text2[score_columns].sum()
        
        # Comparar las puntuaciones y registrar la preferencia
        if score_text1 > score_text2:
            pairwise_data.append({
                'user': user,
                'preferred_text': text1['story_id'],
                'other_text': text2['story_id'],
                'label': 1
            })
            pairwise_data.append({
                'user': user,
                'preferred_text': text2['story_id'],
                'other_text': text1['story_id'],
                'label': 0
            })
        elif score_text1 < score_text2:
            pairwise_data.append({
                'user': user,
                'preferred_text': text2['story_id'],
                'other_text': text1['story_id'],
                'label': 1
            })
            pairwise_data.append({
                'user': user,
                'preferred_text': text1['story_id'],
                'other_text': text2['story_id'],
                'label': 0
            })
        else:
            ties_count += 1
        pairs_count += 1
    
    # Comprobar si se generaron los pares esperados
    print(f"Usuario: {user}, Pares generados: {pairs_count}, Coincide con lo esperado: {pairs_count == expected_pairs}")

# Convertir los resultados a un DataFrame
pairwise_df_new = pd.DataFrame(pairwise_data)

# Convertir columnas a minúsculas
pairwise_df_new.columns = [col.lower() for col in pairwise_df_new.columns]

# Mostrar número de empates
print(f"Número de empates encontrados: {ties_count}")

# Mostrar el resultado

# Exportar a CSV si es necesario
pairwise_df_new


Usuario: Expert1, Textos: 120, Pares esperados: 7140
Usuario: Expert1, Pares generados: 7140, Coincide con lo esperado: True
Usuario: Expert2, Textos: 120, Pares esperados: 7140
Usuario: Expert2, Pares generados: 7140, Coincide con lo esperado: True
Usuario: Expert3, Textos: 120, Pares esperados: 7140
Usuario: Expert3, Pares generados: 7140, Coincide con lo esperado: True
Usuario: Expert4, Textos: 120, Pares esperados: 7140
Usuario: Expert4, Pares generados: 7140, Coincide con lo esperado: True
Usuario: Expert5, Textos: 120, Pares esperados: 7140
Usuario: Expert5, Pares generados: 7140, Coincide con lo esperado: True
Usuario: Expert6, Textos: 120, Pares esperados: 7140
Usuario: Expert6, Pares generados: 7140, Coincide con lo esperado: True
Número de empates encontrados: 3171


,user,preferred_text,other_text,label
0,Expert1,Después de todo lo que casi hice por ti_gpt4_en,Entre nubes y espejismos_gpt4_en,1
1,Expert1,Entre nubes y espejismos_gpt4_en,Después de todo lo que casi hice por ti_gpt4_en,0
2,Expert1,Después de todo lo que casi hice por ti_gpt4_en,Otro episodio en la lucha de clases_gpt4_en,1
3,Expert1,Otro episodio en la lucha de clases_gpt4_en,Después de todo lo que casi hice por ti_gpt4_en,0
4,Expert1,Después de todo lo que casi hice por ti_gpt4_en,Entre las líneas del destino_gpt4_en,1
...,...,...,...,...
79333,Expert6,Todas las fotos son de cuando éramos jóvenes_p...,Todas las canciones de amor son canciones tris...,0
79334,Expert6,¿Quién mató a Patricio Pron?_patricio,Todas las canciones de amor son canciones tris...,1
79335,Expert6,Todas las canciones de amor son canciones tris...,¿Quién mató a Patricio Pron?_patricio,0
79336,Expert6,¿Quién mató a Patricio Pron?_patricio,Todas las fotos son de cuando éramos jóvenes_p...,1


In [2]:
pairwise_df_new.to_csv('output/pairs_balanced_pronvsprompt.csv')